# Progetto Meta-Token V11: Il Test Definitivo con GPT-2 Medium

Questo notebook rappresenta il culmine del nostro progetto. Dopo aver validato l'architettura con `gpt2-small` e aver riscontrato un "cortocircuito semantico" durante la generazione autonoma, ora testiamo la nostra ipotesi finale: **un modello con maggiore capacità sarà in grado di mantenere la coerenza?**

**L'Upgrade:**

Sostituiamo il modello base con `gpt2-medium` o `GroNLP/gpt2-medium-italian-embeddings` (~355M di parametri), quasi tre volte più grande del precedente. Questo dovrebbe fornire la "potenza" necessaria per gestire simultaneamente:
1. La generazione linguistica coerente.
2. La transizione autonoma tra i ruoli strutturali.
3. La coerenza tra parola e tipo semantico.

Manteniamo la stessa metodologia vincente: generazione di un dataset ibrido di alta qualità e training a due fasi.


### Sezione 0: Setup e Generazione Dati
Installiamo le librerie, configuriamo il device e generiamo al volo un dataset da 5000 esempi per questo esperimento.

In [6]:
#%%
# Installazione delle librerie
!pip install transformers accelerate tqdm -q

import torch
import torch.nn as nn
from torch.nn import functional as F
import json
from pathlib import Path
from tqdm.notebook import tqdm # Usiamo la versione per notebook
from transformers import GPT2Tokenizer, GPT2LMHeadModel, get_linear_schedule_with_warmup
import random
import uuid
import re
from typing import List, Dict, Any

# Configurazione del device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Sto usando il device: {device}")

# Creazione delle cartelle
Path("generator_data_v11").mkdir(exist_ok=True)
Path("model_s1_medium_checkpoints").mkdir(exist_ok=True)
Path("model_s2_medium_final").mkdir(exist_ok=True)

# Scriviamo al volo i file per il generatore (versione ridotta per velocità)
Path("generator_data_v11/vocabs.json").write_text(json.dumps({"OGGETTO_TECNICO": ["server", "drone", "API"], "CONCETTO_SCIENTIFICO": ["gravità", "evoluzione", "DNA"], "PROFESSIONE": ["ingegnere", "artista", "scienziato"], "LINGUAGGIO_PROGRAMMAZIONE": ["Python", "JavaScript", "SQL"], "TEMA_CREATIVO": ["il silenzio", "il futuro", "un ricordo"]}))
Path("generator_data_v11/semantic_map.json").write_text(json.dumps({"PUNCTUATION": {".": "PUNCT", "?": "PUNCT", ",": "PUNCT"}, "ARTICLES": {"un": "ARTICOLO", "il": "ARTICOLO", "la": "ARTICOLO"}, "PREPOSITIONS": {"di": "PREPOSIZIONE", "a": "PREPOSIZIONE", "per": "PREPOSIZIONE"}, "CONJUNCTIONS": {"e": "CONGIUNZIONE", "ma": "CONGIUNZIONE"}, "COMMON_VERBS": {"è": "VERBO_ESSERE", "sono": "VERBO_ESSERE", "ha": "VERBO_AVERE"}, "INTERROGATIVES": {"cosa": "DOMANDA", "come": "DOMANDA"}}))
Path("generator_data_v11/rich_content.json").write_text(json.dumps({"DYNAMIC_EXPLANATION_SCIENTIFIC": ["È un principio fondamentale che governa il nostro universo.", "Si tratta di un meccanismo complesso alla base della vita."], "DYNAMIC_CREATIVE_STORY": ["Un robot sognava di dipingere tramonti che non poteva vedere.", "In una biblioteca, i libri si raccontavano storie a vicenda di notte."], "DYNAMIC_STRUCTURED_DATA": ["- Punto A\n- Punto B", "1. Fase Uno\n2. Fase Due"]}))
Path("generator_data_v11/templates.json").write_text(json.dumps([{"nome": "definizione", "struttura": [["UTENTE", "Spiegami"], ["UTENTE", "CONCETTO_SCIENTIFICO"], ["UTENTE", "?"], ["BOT_RAGIONAMENTO", "Richiesta di definizione."], ["BOT_RISPOSTA", "DYNAMIC_EXPLANATION_SCIENTIFIC"]]}, {"nome": "creativo", "struttura": [["UTENTE", "Racconta una storia su"], ["UTENTE", "TEMA_CREATIVO"], ["UTENTE", "."], ["BOT_RAGIONAMENTO", "Richiesta creativa."], ["BOT_NARRATIVE", "DYNAMIC_CREATIVE_STORY"]]}, {"nome": "multi_turno", "struttura": [["UTENTE", "Vorrei diventare un"], ["UTENTE", "PROFESSIONE"], ["UTENTE", "."], ["BOT_DOMANDA", "Interessante! Quale aspetto ti affascina di più?"], ["UTENTE_FOLLOWUP", "La possibilità di creare cose nuove."], ["BOT_CONSIGLIO", "Allora la pratica è fondamentale."]]}]))

# Classe del Generatore (versione semplificata per il notebook)
class DatasetGenerator:
    def __init__(self, data_path=Path("generator_data_v11")):
        self.vocab = json.load(open(data_path / "vocabs.json"))
        self.rich_content = json.load(open(data_path / "rich_content.json"))
        self.templates = json.load(open(data_path / "templates.json"))
        raw_map = json.load(open(data_path / "semantic_map.json")); self.semantic_map = {k: v for cat in raw_map.values() for k, v in cat.items()}
    def _get_sem(self, w): return self.semantic_map.get(w.lower(), "PAROLA_CONTENUTO")
    def generate_example(self):
        template = random.choice(self.templates); sequence = []
        for ruolo, defi in template['struttura']:
            if defi in self.vocab: sequence.append({"word": random.choice(self.vocab[defi]), "ruolo": ruolo, "semantico": defi})
            elif defi.startswith("DYNAMIC_"):
                content = random.choice(self.rich_content.get(defi, [""]))
                for word in re.split(r'(\s+)', content):
                    if word.strip(): sequence.append({"word": word, "ruolo": ruolo, "semantico": "CONTENUTO_DINAMICO"})
            else:
                for word in defi.split(): sequence.append({"word": word, "ruolo": ruolo, "semantico": self._get_sem(word)})
        return {"id": f"gen_{uuid.uuid4()}", "sequence": sequence}

# Generazione del dataset
print("Generazione del dataset da 5000 esempi...")
generator = DatasetGenerator()
dataset = [generator.generate_example() for _ in tqdm(range(5000))]
DATASET_FILE = Path('dataset_medium.json')
with open(DATASET_FILE, 'w', encoding='utf-8') as f:
    json.dump({"examples": dataset}, f, indent=2, ensure_ascii=False)
print(f"Dataset '{DATASET_FILE}' creato.")



Sto usando il device: cuda
Generazione del dataset da 5000 esempi...


  0%|          | 0/5000 [00:00<?, ?it/s]

Dataset 'dataset_medium.json' creato.


### Sezione 1: Modello, Dati e Funzioni di Training
Definiamo l'architettura `MetaGPT2Autonomous` e le funzioni di supporto. Il codice è quasi identico a prima, ma ora è parametrizzato per caricare `GroNLP/gpt2-medium-italian-embeddings`.

In [7]:
#%%
# --- Definizioni e Configurazioni ---
MODEL_NAME = 'GroNLP/gpt2-medium-italian-embeddings'

# Parametri Stage 1
S1_MAX_ITERS = 2000; S1_LR = 3e-5; S1_WARMUP = 200; S1_PATIENCE = 4
S1_SAVE_DIR = Path("model_s1_medium_checkpoints")

# Parametri Stage 2
S2_MAX_ITERS = 1000; S2_LR = 1e-5; S2_WARMUP = 100; S2_PATIENCE = 4
S2_SAVE_DIR = Path("model_s2_medium_final")

# Parametri Comuni
BATCH_SIZE = 4; BLOCK_SIZE = 256; EVAL_INTERVAL = 100; EVAL_ITERS = 50
ACCUMULATION_STEPS = 4 # Simula un batch size più grande (4*4=16)

# Caricamento Dati
tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME); tokenizer.pad_token = tokenizer.eos_token
raw_data = json.load(open(DATASET_FILE, 'r', encoding='utf-8'))['examples']
train_data = raw_data[:-100]; val_data = raw_data[-100:]
all_ruoli = set(['<PAD>'] + [t['ruolo'] for s in raw_data for t in s['sequence']])
all_semantici = set(['<PAD>'] + [t['semantico'] for s in raw_data for t in s['sequence']])
ruolo_to_id = {r: i for i, r in enumerate(sorted(list(all_ruoli)))}; id_to_ruolo = {i: r for i, r in enumerate(sorted(list(all_ruoli)))}
semantico_to_id = {s: i for i, s in enumerate(sorted(list(all_semantici)))}; id_to_semantico = {i: s for i, s in enumerate(sorted(list(all_semantici)))}

def get_batch(split, batch_size):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data), (batch_size,)); seqs_json = [data[i]['sequence'] for i in ix]
    input_ids, ruolo_ids, semantico_ids = [], [], []
    for seq in seqs_json:
        s_i, s_r, s_s = [], [], []
        for t in seq:
            toks = tokenizer.encode(" " + t['word'])
            s_i.extend(toks); s_r.extend([ruolo_to_id.get(t['ruolo'], 0)]*len(toks)); s_s.extend([semantico_to_id.get(t['semantico'], 0)]*len(toks))
        input_ids.append(torch.tensor(s_i)); ruolo_ids.append(torch.tensor(s_r)); semantico_ids.append(torch.tensor(s_s))

    p_i = nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)[:,:BLOCK_SIZE]
    p_r = nn.utils.rnn.pad_sequence(ruolo_ids, batch_first=True, padding_value=ruolo_to_id['<PAD>'])[:,:BLOCK_SIZE]
    p_s = nn.utils.rnn.pad_sequence(semantico_ids, batch_first=True, padding_value=semantico_to_id['<PAD>'])[:,:BLOCK_SIZE]

    return p_i.to(device), p_r.to(device), p_s.to(device), p_i.clone().to(device), p_r.clone().to(device), p_s.clone().to(device)

class MetaGPT2Autonomous(nn.Module):
    def __init__(self, model_name, num_ruoli, num_semantici):
        super().__init__()
        self.gpt2 = GPT2LMHeadModel.from_pretrained(model_name)
        d_model = self.gpt2.config.n_embd
        self.ruolo_embedding_table = nn.Embedding(num_ruoli, d_model)
        self.semantico_embedding_table = nn.Embedding(num_semantici, d_model)
        self.lm_head_ruolo = nn.Linear(d_model, num_ruoli)
        self.lm_head_semantico = nn.Linear(d_model, num_semantici)

    def forward(self, word_idx, ruolo_idx, semantico_idx, labels_w=None, labels_r=None, labels_s=None, stage1=False):
        word_embeds = self.gpt2.transformer.wte(word_idx)
        ruolo_embeds = self.ruolo_embedding_table(ruolo_idx)
        semantico_embeds = self.semantico_embedding_table(semantico_idx)
        inputs_embeds = word_embeds + ruolo_embeds + semantico_embeds
        attention_mask = (word_idx != tokenizer.pad_token_id).float()

        if stage1:
            outputs = self.gpt2(inputs_embeds=inputs_embeds, attention_mask=attention_mask, labels=labels_w)
            return None, None, None, outputs.loss

        transformer_outputs = self.gpt2.transformer(inputs_embeds=inputs_embeds, attention_mask=attention_mask)
        hidden_states = transformer_outputs.last_hidden_state
        logits_w = self.gpt2.lm_head(hidden_states)
        logits_r = self.lm_head_ruolo(hidden_states)
        logits_s = self.lm_head_semantico(hidden_states)

        loss = None
        if labels_w is not None:
            loss_weights = {'word': 1.0, 'ruolo': 0.5, 'semantico': 0.5}
            s_lw, l_w = logits_w[..., :-1, :].contiguous(), labels_w[..., 1:].contiguous()
            s_lr, l_r = logits_r[..., :-1, :].contiguous(), labels_r[..., 1:].contiguous()
            s_ls, l_s = logits_s[..., :-1, :].contiguous(), labels_s[..., 1:].contiguous()
            loss_w = F.cross_entropy(s_lw.view(-1, s_lw.size(-1)), l_w.view(-1), ignore_index=tokenizer.pad_token_id)
            loss_r = F.cross_entropy(s_lr.view(-1, s_lr.size(-1)), l_r.view(-1), ignore_index=ruolo_to_id['<PAD>'])
            loss_s = F.cross_entropy(s_ls.view(-1, s_ls.size(-1)), l_s.view(-1), ignore_index=semantico_to_id['<PAD>'])
            loss = loss_weights['word'] * loss_w + loss_weights['ruolo'] * loss_r + loss_weights['semantico'] * loss_s
        return logits_w, logits_r, logits_s, loss

    @torch.no_grad()
    def generate(self, word_idx, ruolo_idx, semantico_idx, max_new_tokens, top_p=0.95, repetition_penalty=1.2):
        self.eval()
        for _ in range(max_new_tokens):
            w_c, r_c, s_c = word_idx[:, -BLOCK_SIZE:], ruolo_idx[:, -BLOCK_SIZE:], semantico_idx[:, -BLOCK_SIZE:]
            lw, lr, ls, _ = self(w_c, r_c, s_c)
            def sample(logits, is_word=False):
                logits = logits[:, -1, :]
                if is_word and repetition_penalty > 1.0:
                    for i in range(word_idx.shape[0]):
                        unique_toks = torch.unique(word_idx[i]); logits[i, unique_toks] /= repetition_penalty
                probs = F.softmax(logits, dim=-1); sorted_probs, s_i = torch.sort(probs, descending=True); cum_probs = torch.cumsum(sorted_probs, dim=-1)
                s_i_rem = cum_probs > top_p; s_i_rem[..., 1:] = s_i_rem[..., :-1].clone(); s_i_rem[..., 0] = 0
                i_rem = s_i_rem.scatter(1, s_i, s_i_rem); probs[i_rem] = 0; probs /= (probs.sum(dim=-1, keepdim=True) + 1e-9)
                return torch.multinomial(probs, num_samples=1)
            next_w = sample(lw, is_word=True); next_r = sample(lr); next_s = sample(ls)
            word_idx, ruolo_idx, semantico_idx = torch.cat((word_idx, next_w), dim=1), torch.cat((ruolo_idx, next_r), dim=1), torch.cat((semantico_idx, next_s), dim=1)
        self.train()
        return word_idx, ruolo_idx, semantico_idx



tokenizer_config.json:   0%|          | 0.00/135 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

### Sezione 2: Stage 1 - Fine-tuning Linguistico (`GroNLP/gpt2-medium-italian-embeddings`)
Iniziamo il training. Questa fase potrebbe richiedere un po' di tempo (circa 20-30 minuti) a seconda della GPU assegnata da Colab.

In [ ]:
#%%
@torch.no_grad()
def estimate_loss_stage1(model):
    out = {}; model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(EVAL_ITERS)
        for k in range(EVAL_ITERS):
            xb_w, xb_r, xb_s, yb_w, _, _ = get_batch(split, BATCH_SIZE); _, _, _, loss = model(xb_w, xb_r, xb_s, labels_w=yb_w, stage1=True); losses[k] = loss.item()
        out[split] = losses.mean()
    model.train(); return out

print("\n--- INIZIO STAGE 1: FINE-TUNING LINGUISTICO ---")
stage1_model = MetaGPT2Autonomous(MODEL_NAME, len(ruolo_to_id), len(semantico_to_id)).to(device)
optimizer = torch.optim.AdamW(stage1_model.parameters(), lr=S1_LR)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=S1_WARMUP, num_training_steps=S1_MAX_ITERS)

best_val_loss = float('inf'); patience_counter = 0
progress_bar = tqdm(range(S1_MAX_ITERS), desc="Stage 1")

for iter in progress_bar:
    if iter % EVAL_INTERVAL == 0 or iter == S1_MAX_ITERS - 1:
        losses = estimate_loss_stage1(stage1_model); val_loss = losses['val']; lr = scheduler.get_last_lr()[0]
        progress_bar.set_description(f"S1 Iter {iter}: Val loss {val_loss:.4f}, LR {lr:.6f}")
        if val_loss < best_val_loss:
            best_val_loss = val_loss; patience_counter = 0; torch.save(stage1_model.state_dict(), S1_SAVE_DIR / 'best_model.pt')
        else:
            patience_counter += 1
        if patience_counter >= S1_PATIENCE: print("Early stopping in Stage 1."); break

    # Gradient Accumulation
    for i in range(ACCUMULATION_STEPS):
        xb_w, xb_r, xb_s, yb_w, _, _ = get_batch('train', BATCH_SIZE); _, _, _, loss = stage1_model(xb_w, xb_r, xb_s, labels_w=yb_w, stage1=True); loss = loss / ACCUMULATION_STEPS; loss.backward()

    optimizer.step(); scheduler.step(); optimizer.zero_grad()

print("--- STAGE 1 COMPLETATO ---")



--- INIZIO STAGE 1: FINE-TUNING LINGUISTICO ---


model.safetensors:   0%|          | 0.00/1.36G [00:00<?, ?B/s]

Stage 1:   0%|          | 0/2000 [00:00<?, ?it/s]

### Sezione 3: Stage 2 - Fine-tuning per l'Autonomia (`gpt2-medium`)
Carichiamo il modello migliore e iniziamo la fase finale per insegnargli a predire i meta-token.


In [4]:
#%%
@torch.no_grad()
def estimate_loss_stage2(model):
    out = {}; model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(EVAL_ITERS)
        for k in range(EVAL_ITERS):
            xb_w, xb_r, xb_s, yb_w, yb_r, yb_s = get_batch(split, BATCH_SIZE); _, _, _, loss = model(xb_w, xb_r, xb_s, yb_w, yb_r, yb_s); losses[k] = loss.item()
        out[split] = losses.mean()
    model.train(); return out

print("\n--- INIZIO STAGE 2: FINE-TUNING PER L'AUTONOMIA ---")
stage2_model = MetaGPT2Autonomous(MODEL_NAME, len(ruolo_to_id), len(semantico_to_id))
print(f"Caricamento pesi da checkpoint Stage 1: {S1_SAVE_DIR / 'best_model.pt'}")
stage2_model.load_state_dict(torch.load(S1_SAVE_DIR / 'best_model.pt', map_location=device), strict=False); stage2_model.to(device)

optimizer = torch.optim.AdamW(stage2_model.parameters(), lr=S2_LR)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=S2_WARMUP, num_training_steps=S2_MAX_ITERS)

best_val_loss = float('inf'); patience_counter = 0
progress_bar = tqdm(range(S2_MAX_ITERS), desc="Stage 2")

for iter in progress_bar:
    if iter % EVAL_INTERVAL == 0 or iter == S2_MAX_ITERS - 1:
        losses = estimate_loss_stage2(stage2_model); val_loss = losses['val']; lr = scheduler.get_last_lr()[0]
        progress_bar.set_description(f"S2 Iter {iter}: Val loss {val_loss:.4f}, LR {lr:.6f}")
        if val_loss < best_val_loss:
            best_val_loss = val_loss; patience_counter = 0; torch.save(stage2_model.state_dict(), S2_SAVE_DIR / 'best_model.pt')
        else:
            patience_counter += 1
        if patience_counter >= S2_PATIENCE: print("Early stopping in Stage 2."); break

    # Gradient Accumulation
    for i in range(ACCUMULATION_STEPS):
        xb_w, xb_r, xb_s, yb_w, yb_r, yb_s = get_batch('train', BATCH_SIZE); _, _, _, loss = stage2_model(xb_w, xb_r, xb_s, yb_w, yb_r, yb_s); loss = loss / ACCUMULATION_STEPS; loss.backward()

    optimizer.step(); scheduler.step(); optimizer.zero_grad()

print("--- STAGE 2 COMPLETATO ---")




--- INIZIO STAGE 2: FINE-TUNING PER L'AUTONOMIA ---
Caricamento pesi da checkpoint Stage 1: model_s1_medium_checkpoints/best_model.pt


Stage 2:   0%|          | 0/1000 [00:00<?, ?it/s]

--- STAGE 2 COMPLETATO ---


### Sezione 4: Generazione Finale e Analisi
Il momento della verità. Vediamo se la maggiore capacità di `gpt2-medium` risolve il problema della coerenza.

In [5]:
#%%
print("\n" + "="*20 + " GENERAZIONE FINALE " + "="*20)
print("Caricamento del modello migliore per la generazione...")

final_model = MetaGPT2Autonomous(MODEL_NAME, len(ruolo_to_id), len(semantico_to_id))
final_model.load_state_dict(torch.load(S2_SAVE_DIR / 'best_model.pt', map_location=device)); final_model.to(device)

start_text = "Spiegami il concetto di buco nero"
start_ruolo = 'UTENTE'; start_semantico = 'PAROLA_CONTENUTO'

prompt_tokens = tokenizer.encode(" " + start_text)
context_w = torch.tensor([prompt_tokens], dtype=torch.long, device=device)
context_r = torch.full_like(context_w, ruolo_to_id.get(start_ruolo, 0))
context_s = torch.full_like(context_w, semantico_to_id.get(start_semantico, 0))

generated_w, generated_r, generated_s = final_model.generate(context_w, context_r, context_s, max_new_tokens=100)

print(f"Prompt: '{start_text}'\n")
print("Output generato (con meta-token predetti):\n")
print(f"{'Parola':<25} | {'Ruolo Preditto':<25} | {'Semantica Predetta'}")
print("-" * 80)

full_sequence_ids = generated_w[0].tolist(); generated_text = ""
for i in range(len(prompt_tokens), len(full_sequence_ids)):
    word = tokenizer.decode([full_sequence_ids[i]])
    ruolo = id_to_ruolo.get(generated_r[0, i].item(), "N/A")
    semantico = id_to_semantico.get(generated_s[0, i].item(), "N/A")
    print(f"{word:<25} | {ruolo:<25} | {semantico}")
    generated_text += word

print("\n--- Testo Generato Pulito ---")
print(start_text + " " + generated_text.strip())


==================== GENERAZIONE FINALE ====================
Caricamento del modello migliore per la generazione...
Prompt: 'Spiegami il concetto di buco nero'

Output generato (con meta-token predetti):

Parola                    | Ruolo Preditto            | Semantica Predetta
--------------------------------------------------------------------------------
 un                       | UTENTE                    | PAROLA_CONTENUTO
a                         | UTENTE                    | PAROLA_CONTENUTO
 grav                     | UTENTE                    | PAROLA_CONTENUTO
ent                       | UTENTE                    | PAROLA_CONTENUTO
are                       | UTENTE                    | PAROLA_CONTENUTO
 ev                       | UTENTE                    | PAROLA_CONTENUTO
oria                      | UTENTE                    | PAROLA_CONTENUTO
 su                       | UTENTE                    | CONCETTO_SCIENTIFICO
ol                        | UTENTE                